# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [12]:
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [38]:
clicks = df[df.action == 'click']['id'].unique()
views = df[df.action == 'view']['id'].unique()

print("Number of Clicks: {}     Number of Views: {}".format(len(clicks), len(views)))
print("Number of viewers who didn't click: {}".format(len(views) - len(clicks)))

Number of Clicks: 1860     Number of Views: 6328
Number of viewers who didn't click: 4468


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
"""
We are measuring frequency of occurence so I will use the Chi Squared analysis

H0: The experimental group will have a higher click frequency
H1: The experimental group will not have a higher click frequency

Using 1-sides test
"""

In [60]:
df['count'] = 1

In [63]:
ctrl = df[df.group=='control'].pivot(index='id', columns='action', values='count').fillna(0)
expr = df[df.group=='control'].pivot(index='id', columns='action', values='count').fillna(0)

In [66]:
expr

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [42]:
con_tab = pd.crosstab(df.group, df.action)

In [40]:
import scipy.stats as stats

In [45]:
p

0.056537191086915774

In [49]:
con_tab

action,click,view
group,,
control,932,3332
experiment,928,2996


In [84]:
c, p, dof, expctd = stats.chi2_contingency(con_tab)
p

0.056537191086915774

In [56]:
c_c = con_tab.loc['control'][0]
c_v = con_tab.loc['control'][1]
e_c = con_tab.loc['experiment'][0]
e_v = con_tab.loc['experiment'][1]

control_total =  c_v
control_click_percent_raw = (c_c / c_v)
control_click_percent = (c_c / c_v) * 100

expr_total =  e_v
expr_click_percent = (e_c / e_v)
expr_click_percent = (e_c / e_v) * 100

control_click_percent, expr_click_percent

(27.971188475390157, 30.97463284379172)

In [ ]:
"""
Looks like there was approximately a 3% higher chance of clicking for the experimental group, p value is 0.056 so it is 
technically just under the threshold of being able to reject the null hypothesis
"""

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [77]:
expected = (expr_total * control_click_percent_raw)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [72]:
import numpy as np

In [79]:
diff = (e_c - expected)

In [81]:
n = len(expr)
p = ctrl.click.mean()
var = n * p * (1 - p)
std = np.sqrt(var)
z_score = diff / std
z_score

3.4729643112857396

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [82]:
stats.norm.sf(z_score)

0.00025737189895806245

In [ ]:
"""
The conclusion was the same but the p-value was signifigantly lower. THis indicates that this experimental design is superior
"""

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.